In [2]:
'''
Test for normalize.py
'''



'\nTest for normalize.py\n'

In [69]:
#!/usr/bin/python
import sys, getopt, os
import pandas as pd
import numpy as np

from analytical_settings import HAP1_control_1, HAP1_control_2, HAP1_control_3, HAP1_control_4
from analytical_settings import grouping
from analytical_settings import cauchy_compatibility

from global_functions import read_csv, write_csv
from create_phenosaurus_reference import build_phenosaurus_reference


##################################
# Function 'normalize_to_median'
##################################
# Arguments: reference <pandas DataFrame with accumulated counts sense and antisense>
#            sample    <pandas DataFrame with sense and antisense counts for
#                       sample that requires normalization>
# Returns:   ns        <pandas DataFrame with identical structure to sample but with normalized counts>
##################################
def normalize_to_median(raw, reference):
    # Calculate sense/total ratio for reference dataset and store in 'ref_ratio' column in DataFrame
    reference = reference.assign(ref_ratio=(reference['sense_reference'] + 1)
                                           / (reference['sense_reference'] + reference['antisense_reference'] + 2))
    normalized_data = pd.DataFrame(columns=['gene'])
    if cauchy_compatibility:
        print("original Cauchy.R algorithm...", end='')
    else:
        print("latest algorithm...", end='')
    subset = ['gene', 'sense', 'antisense']
    s = pd.merge(reference, raw, on='gene', how='outer')

    # Split sample(s) into part of sample that can be normalized (sn) and the part the cannot be normalized (scn)
    sn = s[((s['sense'] + s['antisense']) > 0) & (
    (s.sense_reference + s.antisense_reference) > 0)]  # meaning counts in reference and sample
    scn = s[((s['sense'] + s['antisense']) > 0) & (
    (s.sense_reference + s.antisense_reference) == 0)]  # meaning counts in sample but not in reference

    # Calculate sense-ratio for current sample and sort DataFrame based on sense-ratio of reference set
    sn = sn.assign(ratio=(s[subset[1]] + 1) / (s[subset[1]] + s[subset[2]] + 2))
    sn = sn.sort_values(by='ref_ratio')

    # Create empty dataframe for the normalized sample (ns)
    ns = pd.DataFrame(columns=subset)

    if not cauchy_compatibility:

        # Calculate optimal grouping value based on requested grouping (to create equal groups and hence identical normalization for every gene)
        divider = np.round(sn.shape[0] / grouping)
        groupsize = (sn.shape[0] / divider)

        # Group sorted DataFrame in equally sized groups and loop over these groups
        for g, df in sn.groupby(np.arange(len(sn)) // groupsize):
            ref_median = df['ref_ratio'].median()
            sample_median = df['ratio'].median()
            df = df.assign(norm_sense=np.where(df['ratio'] <= sample_median
                                               , np.where(
                    np.round((df['ratio'] / (sample_median) * ref_median * (df[subset[1]] + df[subset[2]]))) > (
                    df[subset[1]] + df[subset[2]])  # ie. the normalized sense count is higer than the total counts
                    , (df[subset[1]] + df[subset[2]])
                    , np.round((df['ratio'] / (sample_median) * ref_median * (df[subset[1]] + df[subset[2]]))))
                                               , np.where(np.round(((1 - (1 - df['ratio']) / (1 - sample_median) * (
                1 - ref_median)) * (df[subset[1]] + df[subset[2]]))) > (df[subset[1]] + df[subset[2]])
                                                          , (df[subset[1]] + df[subset[2]])
                                                          , np.round(((1 - (1 - df['ratio']) / (1 - sample_median) * (
                    1 - ref_median)) * (df[subset[1]] + df[subset[2]]))))))

            df = df.assign(norm_antisense=(df[subset[1]] + df[subset[2]] - df['norm_sense']))
            df[['norm_sense', 'norm_antisense']] = df[['norm_sense', 'norm_antisense']].astype(np.int32)
            df = df[['gene', 'norm_sense', 'norm_antisense']]
            df.columns = subset
            print("\nDataframe df:\n%s" % (df))
            print("Column names (subset):\n%s" % (subset))
            print("\nns ns:\n%s" %(ns))
            ns = ns.append(df, ignore_index=True)  # Append to DataFrame with normalized counts of previous group

    else:
        # If cauchy_compatibility==True, grouping of genes will be performed according to the previous version of
        # the pipeline as in 'cauchy.R'

        # Calculate the number of groups
        divider = int(np.ceil(sn.shape[0] / grouping))  # The total number of groups
        genes = sn.shape[0]

        i = 1  # Current index of input dataframe
        for g in range(0, divider):  # Loop over the groups of genes
            df = pd.DataFrame()  # Create temporary dataframe for current group
            for gi in range(0, grouping):  # Append the genes to the current group
                i = gi + g * grouping  # DataFrame Index
                if i <= genes:  # For the last group
                    df = df.append(sn[i:i + 1])
            ref_median = df['ref_ratio'].median()
            sample_median = df['ratio'].median()
            df = df.assign(norm_sense=np.where(df['ratio'] <= sample_median
                                               , np.where(
                    np.round((df['ratio'] / (sample_median) * ref_median * (df[subset[1]] + df[subset[2]]))) > (
                    df[subset[1]] + df[subset[2]])  # ie. the normalized sense count is higer than the total counts
                    , (df[subset[1]] + df[subset[2]])
                    , np.round((df['ratio'] / (sample_median) * ref_median * (df[subset[1]] + df[subset[2]]))))
                                               , np.where(np.round(((1 - (1 - df['ratio']) / (1 - sample_median) * (
                1 - ref_median)) * (df[subset[1]] + df[subset[2]]))) > (df[subset[1]] + df[subset[2]])
                                                          , (df[subset[1]] + df[subset[2]])
                                                          , np.round(((1 - (1 - df['ratio']) / (1 - sample_median) * (
                    1 - ref_median)) * (df[subset[1]] + df[subset[2]]))))))

            df = df.assign(norm_antisense=(df[subset[1]] + df[subset[2]] - df['norm_sense']))
            df = df[['gene', 'norm_sense', 'norm_antisense']]
            df.columns = subset
            print(df)
            #ns = ns.append(df)  # Append to DataFrame with normalized counts of previous group

    # Merge the normalized sample with the datapoint of the sample that could not be normalized because of absense
    # of these genes in the reference
    #ns = ns.append(scn[subset])
    #normalized_data = pd.merge(ns, normalized_data, on='gene', how='outer')
    #normalized_data[['sense', 'antisense']] = normalized_data[['sense', 'antisense']].astype(np.int32)

    #return normalized_data
    pass


def process_controls(controls):
    controls = controls.copy()
    reference_cols = ['gene', 'sense_reference', 'antisense_reference']
    reference = pd.DataFrame(columns=reference_cols)
    for c, f in controls.items():  # c is control, f = file
        controls[c] = pd.read_csv(f, sep='\t')[['gene', 'sense',
                                                'antisense']]  # Read the file and overwrite the value of the name of the with the actual data
        reference = pd.merge(controls[c], reference, on='gene', how='outer').fillna(0)
        reference = reference.assign(
            sense_reference=(reference[['sense', 'sense_reference']].sum(axis=1)).astype(np.int32))
        reference = reference.assign(
            antisense_reference=(reference[['antisense', 'antisense_reference']].sum(axis=1)).astype(np.int32))
        reference.drop(['sense', 'antisense'], axis=1, inplace=True)
    return controls, reference


def main(data=None):
    '''
    syntax = 'normalize.py -n <controls or directory of replicate> -o <directory where normalized control will be saved>'
    try:
        opts, args = getopt.getopt(argv, "hnosr:", ["normalize=", "outdir=", "screenname=", "refname="])
    except getopt.GetoptError:
        print(syntax)
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print(syntax)
            sys.exit()
        elif opt in ("-n", "--normalize"):
            data = arg
        elif opt in ("-o", "--outdir"):
            outdir = arg
        elif opt in ("-s", "--screenname"):
            screenname = arg
        elif opt in ("-r", "--refname"):
            refname = arg
        else:
            assert False, "unhandled option"
    '''    
    # Create dictionary to store the names of the controls (keys) and their associated records (values)
    control_files = {'HAP1_control_1': HAP1_control_1, 'HAP1_control_2': HAP1_control_2, 'HAP1_control_3': HAP1_control_3,
                'HAP1_control_4': HAP1_control_4}
    # Read in controls and calculate aggregate reference counts
    controls, reference = process_controls(control_files)

    if data == 'controls':
        for c, d in controls.items():  # c = control, d = data
            print("Normalizing control using ", end='')
            norm = normalize_to_median(d, reference).sort_values(by='gene')
            write_csv(''.join([screenname, "_replicate_", str(int(c[-1:])), "_sense_vs_antisense_counts_normalized.csv"]), os.path.join(outdir, ''), norm) # Create file for normalization of samples
            print(control_files[c])
            build_phenosaurus_reference(norm, read_csv(control_files[c]), screenname, refname, outdir, int(c[-1:]))
            print("done")
    else:
        data = os.path.join(data, '')
        raw = read_csv(''.join([data, 'sense_vs_antisense_counts.csv']))[['gene', 'sense', 'antisense']]
        print("Normalizing sample using ", end='')
        norm = normalize_to_median(raw, reference).sort_values(by='gene')
        write_csv('sense_vs_antisense_counts_normalized.csv', data, norm)
        print("done")


In [30]:
#main(data='/media/analyzed_data/synthetic_lethal_screens/PTAR1_output/replicate_1/GRCh38-NCBI_RefSeq/')
data='/media/analyzed_data/synthetic_lethal_screens/PTAR1_output/replicate_1/GRCh38-NCBI_RefSeq/'
data = os.path.join(data, '')
control_files = {'HAP1_control_1': HAP1_control_1, 'HAP1_control_2': HAP1_control_2, 'HAP1_control_3': HAP1_control_3,
                'HAP1_control_4': HAP1_control_4}
controls, reference = process_controls(control_files)
raw = read_csv(''.join([data, 'sense_vs_antisense_counts.csv']))[['gene', 'sense', 'antisense']]

In [70]:
norm = normalize_to_median(raw, reference)

latest algorithm...
Dataframe df:
         gene  sense  antisense
11835   RPL37      5        306
11814   RPL21      8        439
11887   RPS4X     15        320
11804   RPL11      8        288
11811   RPL18      7        250
11807  RPL13A     19        702
2513    CDC27     21        548
11843    RPL6      1         72
6123    HCFC1      9        171
11817   RPL23      8        223
11886   RPS3A      9        228
11889    RPS6      8        214
11838   RPL39     10        347
11848    RPL9      7        229
11844    RPL7      3        126
10550  POLR2A     22        304
11812  RPL18A     13        376
6466    HSPA9     16        324
6785    INTS9     26        747
10084    PES1     12        198
14587    TPX2     12        385
4374    EIF3E     19        384
11830   RPL34      7        262
11826    RPL3     11        312
6727     IMP4     12        103
2568     CDK1     14        290
11385   RBBP6      6        151
11802   RPL10     18        300
14045   THG1L      2         58
8108  


Dataframe df:
           gene  sense  antisense
11115     PTRH2      3          1
4749    FAM101B      2          2
11218     RAB34      2         10
12018    S100A8      0          2
16368   DEFB112      2          1
15909    ZNF257      0          1
16367    CYP4B1      1          0
5703      GLYAT      1          1
3746      DDX23      1          5
5098       FCAR      1          0
12863    SLC5A4      5          2
16430     PRRT3      1          1
934      ARL13A      4          4
13391     SSTR2      2          1
13397      SSX3      1          1
7956       MAFG      0          1
1937    C7orf49      4          4
7576     LILRB2      0          1
2094       CAMP      3          1
16554    GGTLC2      1          1
7482       LCN1      3          2
16621     MUCL1      1          0
8904      NAA11      1          0
2203       CBX1      6          3
8772       MUC7      3          2
16663      RAX2      0          1
13965     TEX30      6          3
16790     HTR3A      1          0


Dataframe df:
            gene  sense  antisense
204        ACTN3     27         28
10264      PILRA     14         17
16235    ZNF780A     18         23
15604      YIPF3     75         60
3736        DDTL     21         15
4652       ETV3L      7          3
4789     FAM133A      6          2
3521       CXCR5      6          2
9620        ODAM     13          7
5413      GABRA3      7          3
1809   C20orf203     18         19
3977       DNAH5     17         11
5434     GADD45G      8          4
5694      GLT6D1      1          2
7338       KNTC1    360        422
15086       UGP2   1522       1907
1982     C9orf43    160        191
12263     SEMA6A    168        172
16311    ZSCAN20     61         72
5075      FBXO40     51         49
16275     ZNF860     69         69
5824        GPC3   6991       7896
2007       CAAP1    348        367
14736     TROVE2    250        244
13219      SPCS1    103         78
9604        OAZ1    101         78
16053     ZNF519    148        174
15845


Dataframe df:
             gene  sense  antisense
13483       STK31    246        259
6582       IFT172    399        403
10046       PDS5B    557        529
7968        MAGIX    201        201
11247       RAB8A     91        105
7120      KHDRBS3    712        629
5515         GBE1    238        221
15128       UNC5C    661        690
2889         CILP     43         45
423        AHCYL2   1340       1313
6451       HSF2BP    190        181
429          AHRR    575        540
14682      TRIM54   1012        998
49          ABCB9     35         42
1198       ATP8A1   3335       3239
10650       PPM1B    767        648
10240        PIGV    239        227
7464        LASP1    290        272
1400        BFSP1    731        721
13482        STK3    284        301
9344       NOTCH4    128        151
1084         ATF1    269        257
353        ADRA1A    104         96
12045      SAMD14     49         55
4011      DNAJC22    141        150
994        ARRDC5     16         17
3935         


Dataframe df:
            gene  sense  antisense
5512         GBA     16         20
10768     PRDM16    376        340
2881        CIB1     56         48
2727        CETP    112         94
7940      MAATS1    114        108
2195        CBR3    130        111
5580        GFAP     18         20
11457       RCC2     45         53
14569       TPM4    546        434
12873    SLC6A15     23         26
943       ARL2BP    102         92
11721     RNF187    117         88
1267     B4GALT1    278        244
4599      ERICH5    166        146
2845       CHRM3     72         60
10249    PIK3C2B     12         11
6341    HNRNPUL2    112         93
13938      TERB2     17         10
9961       PCSK7     13         15
14301    TMEM234     62         47
15491    WHSC1L1    187        176
3191       COPRS    112         93
14039      THEGL     71         34
131        ACBD5    247        237
7768        LRP2     21         26
2854      CHRNA7     94         96
13031     SMURF1   1177       1053
11587

In [59]:
df1 = pd.DataFrame(columns=['A', 'B', 'C'])
df2 = pd.DataFrame({'A':[1,2,3], 'B':[4,5,6], 'C':[7,8,9]})

In [63]:
ns

NameError: name 'ns' is not defined